## Feature Extraction of People

This notebook extracts skills and project involvement from a knowledge graph and saves them as new nodes and relationships. Moreover, the code creates a vector index within neo4j.

You need a .env file which is configured with your neo4j credentials (see knowledge_graph_construction.ipynb) and your OpenAI API key.

In [1]:
import os
from dotenv import load_dotenv
import json
from neo4j import GraphDatabase
#from langchain.prompts import PromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain_openai import ChatOpenAI
import openai
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Neo4jVector


In [2]:
# Load environment variables
load_dotenv()

True

#### 1. Configure OpenAI API and neo4j

In [3]:
# OpenAI API configuration
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
# Neo4j configuration & constraints
neo4j_uri = os.getenv("NEO4J_URI")
neo4j_user = os.getenv("NEO4J_USERNAME")
neo4j_password = os.getenv("NEO4J_PASSWORD")
driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))

#### 2. Function to fetch all employee IDs

In [5]:
# Function to fetch all employee IDs
def fetch_all_employees(session):
    query = """
    MATCH (p:Person)
    RETURN p.id AS employee_id
    """
    result = session.run(query)
    return [record['employee_id'] for record in result]

#### 3. Function to fetch employee data

In [6]:
# Function to fetch employee data
def fetch_employee_data(employee_id, session):
    query = """
    MATCH (p:Person {id: $employee_id})
    OPTIONAL MATCH (p)-[:BELONGS_TO]->(t:Team),
                   (p)-[:REPORTS_TO]->(manager:Person),
                   (p)-[:WROTE]->(chat:chat_message),
                   (p)-[:RECEIVED]->(recchat:chat_message),
                   (p)-[:WROTE]->(email:email_message),
                   (p)-[:RECEIVED]->(recemail:email_message),
                   (p)-[:CREATED]->(doc:Document)
    WITH p, 
         collect(DISTINCT t) AS teams,
         collect(DISTINCT manager) AS managers,
         collect(DISTINCT chat) AS sentchatMessages,
         collect(DISTINCT recchat) AS receivedchatMessages,
         collect(DISTINCT email) AS sentemailMessages,
         collect(DISTINCT recemail) AS receivedemailMessages,
         collect(DISTINCT doc) AS documents
    RETURN p, teams, managers, sentchatMessages, receivedchatMessages, sentemailMessages, receivedemailMessages, documents
    """
    result = session.run(query, employee_id=employee_id)
    data = [record.data() for record in result]
    return json.dumps(data)  # Convert data to JSON format

In [38]:
# #only relevant for single test run
# # Fetch data for a specific employee (example ID)
# employee_data_json = fetch_employee_data(22)
# print(employee_data_json)

#### 4. Extract skills and project involvment

In [7]:

# Function to call the OpenAI API
llm = ChatOpenAI(
    model="gpt-4o-mini",                
    temperature=0,                
    max_tokens=1500,                
)

def extract_skills_and_projects(employee_data_json):
    data = json.loads(employee_data_json)
    if not data:
        return [], []  # Handle empty data
    employee_data = data[0]  # Assuming single employee data
    prompt_template = """
    Extract the key skills and projects for the following employee data:
    Name: {name}
    Role: {role}
    Team: {team}
    Manager: {manager}
    Sent Chat Messages: {sent_chat}
    Received Chat Messages: {received_chat}
    Sent Email Messages: {sent_email}
    Received Email Messages: {received_email}
    Documents Created: {documents}

    Provide a list of skills and projects in a structured format as follows:
    Skills: skill1, skill2, ...
    Projects: project1, project2, ...
    """

    # Prepare prompt data
    name = employee_data['p']['name']
    role = employee_data['p']['role']
    team = employee_data['teams'][0]['name'] if employee_data['teams'] else "N/A"
    manager = employee_data['managers'][0]['name'] if employee_data['managers'] else "N/A"
    sent_chat = ", ".join([msg['content'] for msg in employee_data['sentchatMessages']])
    received_chat = ", ".join([msg['content'] for msg in employee_data['receivedchatMessages']])
    sent_email = ", ".join([msg['subject_line'] for msg in employee_data['sentemailMessages']])
    received_email = ", ".join([msg['subject_line'] for msg in employee_data['receivedemailMessages']])
    documents = ", ".join([doc['document_name'] for doc in employee_data['documents']])

    # Create the prompt
    prompt = prompt_template.format(
        name=name, role=role, team=team, manager=manager,
        sent_chat=sent_chat, received_chat=received_chat,
        sent_email=sent_email, received_email=received_email,
        documents=documents
    )

    # Generate the skills and projects using the LLM
    response = llm(prompt)
    # Parse the response
    content = response.content
    skills = []
    projects = []
    try:
        lines = content.split('\n')
        for line in lines:
            if line.startswith("Skills:"):
                skills = [skill.strip() for skill in line.replace("Skills:", "").strip().split(',')]
            elif line.startswith("Projects:"):
                projects = [project.strip() for project in line.replace("Projects:", "").strip().split(',')]
    except Exception as e:
        print(f"Error parsing response: {e}")

    return skills, projects


#### 5. Safe skills and projects as nodes/relationships

In [8]:
def save_skills_and_projects(employee_id, skills, projects, session):
    for skill in skills:
        query = """
        MATCH (p:Person {id: $employee_id})
        MERGE (s:Skill {name: $skill})
        MERGE (p)-[:HAS_SKILL]->(s)
        """
        session.run(query, employee_id=employee_id, skill=skill)

    for project in projects:
        query = """
        MATCH (p:Person {id: $employee_id})
        MERGE (pr:Project {name: $project})
        MERGE (p)-[:WORKS_ON]->(pr)
        """
        session.run(query, employee_id=employee_id, project=project)

In [9]:
# Main execution loop

with driver.session() as session:
        #employee_ids = [2] #for testing single IDs
        employee_ids = fetch_all_employees(session) #for entire KG
        for employee_id in employee_ids:
            employee_data_json = fetch_employee_data(employee_id, session)
            skills, projects = extract_skills_and_projects(employee_data_json)
            if skills or projects:
                save_skills_and_projects(employee_id, skills, projects, session)
                print(f"Processed employee ID: {employee_id}")

/var/folders/ky/pkg41pm57tjgqckrqnnwf_080000gn/T/ipykernel_5896/2500548549.py:50: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(prompt)


Processed employee ID: 1
Processed employee ID: 2
Processed employee ID: 4
Processed employee ID: 5
Processed employee ID: 6
Processed employee ID: 7
Processed employee ID: 8
Processed employee ID: 10
Processed employee ID: 12
Processed employee ID: 13
Processed employee ID: 15
Processed employee ID: 16
Processed employee ID: 17
Processed employee ID: 18
Processed employee ID: 19
Processed employee ID: 20
Processed employee ID: 21
Processed employee ID: 22
Processed employee ID: 23
Processed employee ID: 24
Processed employee ID: 25
Processed employee ID: 28
Processed employee ID: 29
Processed employee ID: 30
Processed employee ID: 31


#### 6. Create Vector Index (Hybrid Search)

In [80]:
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-large")

vector_index = Neo4jVector.from_existing_graph(
    embeddings_model,
    search_type="hybrid",  
    node_label="Skill",
    text_node_properties=["name"],
    embedding_node_property="embedding",
    index_name = "skill_index_vector",
    keyword_index_name = "skill_index_fulltext"
)


In [81]:
vector_index_projects = Neo4jVector.from_existing_graph(
    embeddings_model,
    search_type="hybrid",  # Hybrid search allows combining vector and exact queries
    node_label="Project",
    text_node_properties=["name"],
    embedding_node_property="embedding",
    index_name = "project_index_vector",
    keyword_index_name = "project_index_fulltext"
)
